In [ ]:
!pip install mediapipe

# 1. Import

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tqdm.notebook import tqdm

# 2. Keypoints using Mediapipe Holistic

In [2]:
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils # drawing utils
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(model_complexity=0,min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            success, image = cap.read()
            print(type(image))
            
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                continue

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image)
            results_pose = pose.process(image)
            

            # Draw the hand annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            hand_np = []
            if results.multi_hand_landmarks:
                for i in range(len(results.multi_hand_landmarks)):
                    print(i)
                    mp_drawing.draw_landmarks(image,results.multi_hand_landmarks[i],mp_hands.HAND_CONNECTIONS,mp_drawing_styles.get_default_hand_landmarks_style(),mp_drawing_styles.get_default_hand_connections_style())
                    hand_np.append(np.array([[res.x, res.y, res.z] for res in results.multi_hand_landmarks[i].landmark]).flatten() if results.multi_hand_landmarks[i] else np.zeros(21*3))
                    
            if len(hand_np) == 0:
                lh = np.zeros(21*3)
                rh = np.zeros(21*3)
            elif len(hand_np) == 1:
                lh = hand_np[0]
                rh = np.zeros(21*3)
            else:
                lh = hand_np[0]
                rh = hand_np[1]        
                    
            # Draw pose annotations        
            mp_drawing.draw_landmarks(image,results_pose.pose_landmarks,mp_pose.POSE_CONNECTIONS,landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            
            ps = np.array([[res.x, res.y, res.z, res.visibility] for res in results_pose.pose_landmarks.landmark]).flatten() if results_pose.pose_landmarks else np.zeros(33*4)
            
            extracted_keypoints = np.concatenate([ps, lh, rh])
            print("EXTRACTED_KEYPOINTS: ")
            print(len(extracted_keypoints))
            
            # Flip the image horizontally for a selfie-view display.
            cv2.imshow('MediaPipe Result', cv2.flip(image, 1))
            if cv2.waitKey(10) & 0xFF == ord('q'):
                  break
        cap.release()
        cap.destroyAllWindows()

# 3. Extract Keypoints Values

In [3]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Manual Collection

In [3]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('LSTM_data_noface_actions') 
SAVE_PATH = os.path.join('keypoints_noface_actions')

# Actions that we try to detect
actions = np.load('actions_list.npy')

In [4]:
# Thirty videos worth of data
no_sequences = 32

# Videos are going to be 60 frames in length
sequence_length = 60

# 6.* Preprocess Data and Create Labels and Features

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}
print(label_map)

In [ ]:
sequences, labels = [], []
for action in tqdm(label_map.keys()):
    if  os.path.exists(os.path.join(SAVE_PATH, action)):
        for sequence in range(1,no_sequences+1):
            window = []
            last_frame = 0
            for frame_num in range(sequence_length):
                if  os.path.exists(os.path.join(SAVE_PATH, action, str(sequence)+"_"+"{}.npy".format(frame_num))): 
                    res = np.load(os.path.join(SAVE_PATH, action, str(sequence)+"_"+"{}.npy".format(frame_num)))
                    last_frame = frame_num
                else: # if the video length is shorter than sequence length, need to pad
    #                 res = np.zeros((1662,),dtype='float64') # zero padding
                    res = np.load(os.path.join(SAVE_PATH, action, str(sequence)+"_"+"{}.npy".format(last_frame))) # last frame padding
                window.append(res)

            # repeat for aug and mir
            window_aug = []
            last_frame = 0
            for frame_num in range(sequence_length):
                if  os.path.exists(os.path.join(SAVE_PATH, action, str(sequence)+"_"+"{}_AUG.npy".format(frame_num))): 
                    res = np.load(os.path.join(SAVE_PATH, action, str(sequence)+"_"+"{}_AUG.npy".format(frame_num)))
                    last_frame = frame_num
                else: 
                    res = np.load(os.path.join(SAVE_PATH, action, str(sequence)+"_"+"{}_AUG.npy".format(last_frame))) # last frame padding
                window_aug.append(res)
                       
            window_mir = []
            last_frame = 0
            for frame_num in range(sequence_length):
                if  os.path.exists(os.path.join(SAVE_PATH, action, str(sequence)+"_"+"{}_MIR.npy".format(frame_num))): 
                    res = np.load(os.path.join(SAVE_PATH, action, str(sequence)+"_"+"{}_MIR.npy".format(frame_num)))
                    last_frame = frame_num
                else: 
                    res = np.load(os.path.join(SAVE_PATH, action, str(sequence)+"_"+"{}_MIR.npy".format(last_frame))) # last frame padding
                window_mir.append(res)

            sequences.append(window)
            labels.append(label_map[action])
            sequences.append(window_aug)
            labels.append(label_map[action])
            sequences.append(window_mir)
            labels.append(label_map[action])

In [72]:
X = np.array(sequences)

In [ ]:
X.shape

In [74]:
y = to_categorical(labels,num_classes=len(actions)).astype(int)

In [78]:
np.set_printoptions(threshold=1000)

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [ ]:
y_train.sum(axis=0)

In [81]:
np.save(os.path.join("LSTM_data_noface_actions","train_data.npy"),X_train)
np.save(os.path.join("LSTM_data_noface_actions","train_labels.npy"),y_train)
np.save(os.path.join("LSTM_data_noface_actions","test_data.npy"),X_test)
np.save(os.path.join("LSTM_data_noface_actions","test_labels.npy"),y_test)

In [ ]:
del X_train
del X_test
del y_train
del y_test

# 7. Build and Train LSTM Neural Network

In [6]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
import sys

In [ ]:
train_data, train_labels = np.load(os.path.join(DATA_PATH,"train_data.npy")), np.load(os.path.join(DATA_PATH,"train_labels.npy"))
test_data, test_labels = np.load(os.path.join(DATA_PATH,"test_data.npy")), np.load(os.path.join(DATA_PATH,"test_labels.npy"))

print(f"Frame features in train set: {train_data.shape}")

In [ ]:
train_labels.shape

In [9]:
log_dir = os.path.join('Logs',DATA_PATH)
tb_callback = TensorBoard(log_dir=log_dir)

In [10]:
checkpoint_path = os.path.join("Logs",DATA_PATH,'checkpoints','cp-{epoch:04d}.ckpt')
cp_callback = ModelCheckpoint(filepath=checkpoint_path,verbose=1,save_weights_only=True,save_freq=50*14)

In [ ]:
model = Sequential()
model.add(LSTM(16, return_sequences=True, activation='relu', input_shape=(sequence_length,258)))
model.add(LSTM(32, return_sequences=True, activation='relu'))
model.add(LSTM(16, return_sequences=False, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(actions), activation='softmax'))

In [95]:
# opt = SGD(learning_rate=0.01,momentum=0.9)
# opt = Adam(learning_rate=0.0001)
opt = 'Adam'

In [96]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [98]:
model.load_weights('Logs\LSTM_data_noface_seq60_actions\checkpoints\cp-0208.ckpt')

In [12]:
model.fit(train_data, train_labels, epochs=500, callbacks=[cp_callback])

Epoch 1/500
27/27 [==============================] - 8s 223ms/step - loss: 0.2282 - categorical_accuracy: 0.9039
Epoch 2/500
27/27 [==============================] - 6s 224ms/step - loss: 0.1597 - categorical_accuracy: 0.9456
Epoch 3/500
27/27 [==============================] - 6s 238ms/step - loss: 0.1800 - categorical_accuracy: 0.9329
Epoch 4/500
27/27 [==============================] - 7s 241ms/step - loss: 0.1267 - categorical_accuracy: 0.9606
Epoch 5/500
27/27 [==============================] - 6s 232ms/step - loss: 0.1103 - categorical_accuracy: 0.9653
Epoch 6/500
27/27 [==============================] - 7s 246ms/step - loss: 0.1246 - categorical_accuracy: 0.9549
Epoch 7/500
27/27 [==============================] - 6s 234ms/step - loss: 0.1729 - categorical_accuracy: 0.9456
Epoch 8/500
27/27 [==============================] - 6s 232ms/step - loss: 0.2348 - categorical_accuracy: 0.9109
Epoch 9/500
27/27 [==============================] - 6s 232ms/step - loss: 0.6303 - categorical_

27/27 [==============================] - 6s 223ms/step - loss: 2.2653 - categorical_accuracy: 0.0995
Epoch 70/500
27/27 [==============================] - 6s 220ms/step - loss: 2.2605 - categorical_accuracy: 0.1146
Epoch 71/500
27/27 [==============================] - 6s 228ms/step - loss: 2.2593 - categorical_accuracy: 0.1273
Epoch 72/500
27/27 [==============================] - 6s 232ms/step - loss: 2.2542 - categorical_accuracy: 0.1215
Epoch 73/500
27/27 [==============================] - 7s 243ms/step - loss: 2.2525 - categorical_accuracy: 0.1204
Epoch 74/500
27/27 [==============================] - 6s 231ms/step - loss: 2.2510 - categorical_accuracy: 0.1238
Epoch 75/500
27/27 [==============================] - 6s 226ms/step - loss: 2.2501 - categorical_accuracy: 0.1227
Epoch 76/500
27/27 [==============================] - 6s 223ms/step - loss: 2.2505 - categorical_accuracy: 0.1250
Epoch 77/500
27/27 [==============================] - 6s 228ms/step - loss: 2.2451 - categorical_accu

27/27 [==============================] - 6s 222ms/step - loss: 0.8146 - categorical_accuracy: 0.6620
Epoch 136/500
27/27 [==============================] - 6s 214ms/step - loss: 0.7962 - categorical_accuracy: 0.6644
Epoch 137/500
27/27 [==============================] - 6s 223ms/step - loss: 0.8988 - categorical_accuracy: 0.6169
Epoch 138/500
27/27 [==============================] - 6s 225ms/step - loss: 0.8439 - categorical_accuracy: 0.6516
Epoch 139/500
27/27 [==============================] - 6s 235ms/step - loss: 0.8191 - categorical_accuracy: 0.6620
Epoch 140/500
27/27 [==============================] - 6s 227ms/step - loss: 0.8325 - categorical_accuracy: 0.6435
Epoch 141/500
27/27 [==============================] - 6s 232ms/step - loss: 0.7977 - categorical_accuracy: 0.6678
Epoch 142/500
27/27 [==============================] - 6s 221ms/step - loss: 0.8265 - categorical_accuracy: 0.6644
Epoch 143/500
27/27 [==============================] - 6s 217ms/step - loss: 0.7742 - categori

27/27 [==============================] - 6s 239ms/step - loss: 2.2382 - categorical_accuracy: 0.1447
Epoch 204/500
27/27 [==============================] - 6s 217ms/step - loss: 2.2307 - categorical_accuracy: 0.1562
Epoch 205/500
27/27 [==============================] - 6s 214ms/step - loss: 2.2253 - categorical_accuracy: 0.1389
Epoch 206/500
27/27 [==============================] - 6s 224ms/step - loss: 2.2068 - categorical_accuracy: 0.1632
Epoch 207/500
27/27 [==============================] - 6s 222ms/step - loss: 2.1912 - categorical_accuracy: 0.1586
Epoch 208/500
10/27 [==========>...................] - ETA: 3s - loss: 2.1674 - categorical_accuracy: 0.1781
Epoch 208: saving model to Logs\LSTM_data_noface_seq60_16actions\checkpoints\cp-0208.ckpt
27/27 [==============================] - 7s 269ms/step - loss: 2.1802 - categorical_accuracy: 0.1597
Epoch 209/500
27/27 [==============================] - 6s 235ms/step - loss: 2.1549 - categorical_accuracy: 0.1516
Epoch 210/500
27/27 [===

27/27 [==============================] - 6s 220ms/step - loss: 2.3046 - categorical_accuracy: 0.0880
Epoch 270/500
27/27 [==============================] - 6s 235ms/step - loss: 2.3047 - categorical_accuracy: 0.0891
Epoch 271/500
27/27 [==============================] - 6s 222ms/step - loss: 2.3009 - categorical_accuracy: 0.0822
Epoch 272/500
27/27 [==============================] - 6s 234ms/step - loss: 2.3036 - categorical_accuracy: 0.0810
Epoch 273/500
27/27 [==============================] - 6s 240ms/step - loss: 2.3017 - categorical_accuracy: 0.0718
Epoch 274/500
27/27 [==============================] - 6s 218ms/step - loss: 2.3020 - categorical_accuracy: 0.0984
Epoch 275/500
27/27 [==============================] - 6s 222ms/step - loss: 2.3011 - categorical_accuracy: 0.0903
Epoch 276/500
27/27 [==============================] - 6s 232ms/step - loss: 2.3017 - categorical_accuracy: 0.1007
Epoch 277/500
27/27 [==============================] - 6s 224ms/step - loss: 2.3013 - categori

27/27 [==============================] - 6s 225ms/step - loss: 1.3756 - categorical_accuracy: 0.4711
Epoch 338/500

Epoch 338: saving model to Logs\LSTM_data_noface_seq60_16actions\checkpoints\cp-0338.ckpt
27/27 [==============================] - 7s 242ms/step - loss: 1.3491 - categorical_accuracy: 0.4699
Epoch 339/500
27/27 [==============================] - 7s 241ms/step - loss: 1.3452 - categorical_accuracy: 0.4942
Epoch 340/500
27/27 [==============================] - 6s 228ms/step - loss: 1.3438 - categorical_accuracy: 0.4676
Epoch 341/500
27/27 [==============================] - 6s 233ms/step - loss: 1.4466 - categorical_accuracy: 0.4144
Epoch 342/500
27/27 [==============================] - 6s 231ms/step - loss: 1.3489 - categorical_accuracy: 0.4757
Epoch 343/500
27/27 [==============================] - 6s 240ms/step - loss: 1.3015 - categorical_accuracy: 0.4977
Epoch 344/500
27/27 [==============================] - 6s 233ms/step - loss: 1.3085 - categorical_accuracy: 0.5093
Epo

27/27 [==============================] - 7s 280ms/step - loss: 0.6944 - categorical_accuracy: 0.7350
Epoch 405/500
27/27 [==============================] - 7s 242ms/step - loss: 0.7410 - categorical_accuracy: 0.7060
Epoch 406/500
27/27 [==============================] - 6s 230ms/step - loss: 0.6896 - categorical_accuracy: 0.7384
Epoch 407/500
27/27 [==============================] - 6s 237ms/step - loss: 0.6803 - categorical_accuracy: 0.7373
Epoch 408/500
27/27 [==============================] - 6s 231ms/step - loss: 0.6964 - categorical_accuracy: 0.7315
Epoch 409/500
27/27 [==============================] - 6s 234ms/step - loss: 0.6618 - categorical_accuracy: 0.7384
Epoch 410/500
27/27 [==============================] - 6s 238ms/step - loss: 0.6697 - categorical_accuracy: 0.7500
Epoch 411/500
27/27 [==============================] - 6s 237ms/step - loss: 0.6524 - categorical_accuracy: 0.7488
Epoch 412/500
27/27 [==============================] - 6s 239ms/step - loss: 0.6534 - categori

27/27 [==============================] - 7s 244ms/step - loss: 0.4891 - categorical_accuracy: 0.8264
Epoch 471/500
27/27 [==============================] - 6s 237ms/step - loss: 0.4285 - categorical_accuracy: 0.8507
Epoch 472/500
27/27 [==============================] - 6s 237ms/step - loss: 0.4446 - categorical_accuracy: 0.8356
Epoch 473/500
27/27 [==============================] - 7s 248ms/step - loss: 0.4474 - categorical_accuracy: 0.8287
Epoch 474/500
27/27 [==============================] - 6s 224ms/step - loss: 0.4068 - categorical_accuracy: 0.8600
Epoch 475/500
27/27 [==============================] - 6s 215ms/step - loss: 0.3986 - categorical_accuracy: 0.8495
Epoch 476/500
27/27 [==============================] - 6s 225ms/step - loss: 0.5283 - categorical_accuracy: 0.7975
Epoch 477/500
27/27 [==============================] - 6s 236ms/step - loss: 0.5024 - categorical_accuracy: 0.8160
Epoch 478/500
27/27 [==============================] - 6s 227ms/step - loss: 0.4326 - categori

In [ ]:
model.summary()

# 8. Make Predictions

In [ ]:
res = model.predict(test_data)
res.shape

In [ ]:
res[2]

In [100]:
actions[np.argmax(res[2])]

'french fries'

In [101]:
actions[np.argmax(y_test[2])]

'french fries'

# 9. Save Weights

In [102]:
model.save('noface_actions.h5')

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [103]:
del model

In [ ]:
model = tensorflow.keras.models.load_model("noface_actions.h5")

In [50]:
# Save keras model & weights to saved_model folder
tensorflow.saved_model.save(model,'saved_model/13')
# Use this command to convert into onnx to deploy 
!python -m tf2onnx.convert --saved-model C:\Users\ryanr\ASLtoEng\saved_model\13 --output model13.onnx

C:\Users\ryanr\AppData\Local\Programs\Python\Python310\lib\runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-08-26 22:20:35,497 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2022-08-26 22:20:37,137 - INFO - Signatures found in model: [serving_default].
2022-08-26 22:20:37,137 - WARNING - '--signature_def' not specified, using first signature: serving_default
2022-08-26 22:20:37,137 - INFO - Output names: ['dense_3']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2022-08-26 22:20:37,418 - WARNING - From C:\Users\ryanr\AppData\Local\Programs\Python\Python310\lib\site-packages\tf2onnx\tf_loader.py:711: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v

# 10. Evaluation using Confusion Matrix and Accuracy

In [9]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [109]:
accuracy_score(ytrue, yhat)

0.7708333333333334

# 11. Test in Real Time

In [9]:
from scipy import stats

In [10]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num%3], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
        while cap.isOpened():

            # Read feed
            ret, image = cap.read()

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image)
            results_pose = pose.process(image)


            # Draw the hand annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            hand_np = []
            if results.multi_hand_landmarks:
                for i in range(len(results.multi_hand_landmarks)):
                    mp_drawing.draw_landmarks(image,results.multi_hand_landmarks[i],mp_hands.HAND_CONNECTIONS,mp_drawing_styles.get_default_hand_landmarks_style(),mp_drawing_styles.get_default_hand_connections_style())
                    hand_np.append(np.array([[res.x, res.y, res.z] for res in results.multi_hand_landmarks[i].landmark]).flatten() if results.multi_hand_landmarks[i] else np.zeros(21*3))

            if len(hand_np) == 0:
                lh = np.zeros(21*3)
                rh = np.zeros(21*3)
            elif len(hand_np) == 1:
                lh = hand_np[0]
                rh = np.zeros(21*3)
            else:
                lh = hand_np[0]
                rh = hand_np[1]        

            # Draw pose annotations        
            mp_drawing.draw_landmarks(image,results_pose.pose_landmarks,mp_pose.POSE_CONNECTIONS,landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

            ps = np.array([[res.x, res.y, res.z, res.visibility] for res in results_pose.pose_landmarks.landmark]).flatten() if results_pose.pose_landmarks else np.zeros(33*4)

            keypoints = np.concatenate([ps, lh, rh])
            sequence.append(keypoints)
            sequence = sequence[-sequence_length:]

            if len(sequence) == sequence_length:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                predictions.append(np.argmax(res))


                #3. Viz logic
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 

                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 4: 
                    sentence = sentence[-4:]

                # Viz probabilities
                image = prob_viz(res, actions, image, colors)                

            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()